In [1]:
import torch

# CUDAが使えるかどうかを確認
cuda_available = torch.cuda.is_available()

if cuda_available:
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")
     

CUDA is available! GPU: Quadro RTX 5000


# **1枚画像**

In [1]:
from ultralytics import RTDETR
import wget
import cv2
import matplotlib.pyplot as plt

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model = RTDETR(r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt")

# 推論を実行: YOLOの推論
image_path = r'C:\Users\ykita\FacePhoto_instance\201-295\Image\293-20200311-6-095803_287b73c1d0b04ca725de2651547efd9a530231320625fb361bef3708dc3df9ee.jpg'
results = model(image_path, save=False)

for r in results:
    boxes = r.boxes  # YOLOv8の場合

    # YOLO形式出力
    for box in boxes:
        cls_id = int(box.cls[0])  # クラスID
        x_center, y_center, width, height = box.xywhn[0]

        # YOLO形式: class x_center y_center width height
        # 小数点以下6桁で出力したい場合
        print(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    # 検出結果を画像に描画して表示
    # plt.figure(figsize=(12, 8))
    # plt.imshow(cv2.cvtColor(r.plot(), cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()

    # 結果の保存
    #cv2.imwrite('result.jpg', r.plot())

modelが未定義のため、ロードを実行します。


c:\Users\ykita\FacePhoto_instance\instanceenv\lib\site-packages\ultralytics\nn\tasks.py:634: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_locati


image 1/1 C:\Users\ykita\FacePhoto_instance\201-295\Image\293-20200311-6-095803_287b73c1d0b04ca725de2651547efd9a530231320625fb361bef3708dc3df9ee.jpg: 640x640 1 Right_eye, 1 Left_eye, 771.0ms
Speed: 9.3ms preprocess, 771.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
0 0.268958 0.487141 0.170173 0.112953
1 0.687053 0.477957 0.160687 0.102739


In [2]:
from ultralytics import RTDETR
import wget
import cv2
import matplotlib.pyplot as plt
import os

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"
    model = RTDETR(model_path)

# 推論を実行
image_path = r'C:\Users\ykita\FacePhoto_instance\201-295\Image\293-20200311-6-095803_287b73c1d0b04ca725de2651547efd9a530231320625fb361bef3708dc3df9ee.jpg'
results = model(image_path, save=False)

# 保存先のテキストファイルパスを指定
save_txt_path = r'C:\Users\ykita\FacePhoto_instance\result_yolo.txt'

# YOLO形式の推論結果をテキストファイルに書き込み
with open(save_txt_path, 'w') as f:
    for r in results:
        boxes = r.boxes  # YOLOv8系の出力
        for box in boxes:
            cls_id = int(box.cls[0])  
            x_center, y_center, width, height = box.xywhn[0]
            # YOLO形式: class x_center y_center width height （小数点以下6桁）
            line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
            f.write(line)

print(f"YOLO形式の推論結果をテキストファイルに保存しました: {save_txt_path}")



image 1/1 C:\Users\ykita\FacePhoto_instance\201-295\Image\293-20200311-6-095803_287b73c1d0b04ca725de2651547efd9a530231320625fb361bef3708dc3df9ee.jpg: 640x640 1 Right_eye, 1 Left_eye, 879.9ms
Speed: 7.0ms preprocess, 879.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)
YOLO形式の推論結果をテキストファイルに保存しました: C:\Users\ykita\FacePhoto_instance\result_yolo.txt


# **複数画像**

階層構造
```
archive.zip/
   ├── data.yaml  # configuration file
   ├── train.txt  # list of train subset image paths
   │
   ├── images/
   │   ├── train/  # directory with images for train subset
   │   │    ├── image1.jpg
   │   │    ├── image2.jpg
   │   │    ├── image3.jpg
   │   │    └── ...
   ├── labels/
   │   ├── train/  # directory with annotations for train subset
   │   │    ├── image1.txt
   │   │    ├── image2.txt
   │   │    ├── image3.txt
   │   │    └── ...

```

***.txt (ラベル)
```
0 0.260745 0.535950 0.218015 0.030600
1 0.760795 0.533850 0.213035 0.043800
```

daya.yaml
```
names:
  0: Right_eye
  1: Left_eye
path: .
train: train.txt
```

train.txt
```
data/images/train/***.jpg
data/images/train/***.jpg
...(pathの羅列)
```
※ラベルや画像と対応を合わせること



In [1]:
import os
import glob
from ultralytics import RTDETR
from tqdm import tqdm  # tqdmライブラリをインポート

# 1) 画像のフォルダを指定する
orig_image_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\image"
# 2) 出力先のフォルダを指定する。フォルダがなければ作成する
dst_label_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\archive\labels"
os.makedirs(dst_label_dir, exist_ok=True)

# モデルが未定義ならロードする
try:
    model  # 変数modelが定義されているか確認
except NameError:
    print("modelが未定義のため、ロードを実行します。")
    model_path = r"C:\Users\ykita\FacePhoto_instance\models\135best_rtdetr.pt"
    model = RTDETR(model_path)

# 処理対象とする画像の拡張子
image_extensions = [
    '.jpg', '.jpeg', '.JPG', '.JPEG',
    '.png', '.PNG',
    '.tif', '.tiff', '.TIF', '.TIFF'
]

# 指定フォルダ配下の該当画像ファイルをすべて取得
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(orig_image_dir, f'*{ext}')))

# tqdmで進捗状況を表示しながらループ
for image_path in tqdm(image_files, desc='推論中', unit='枚'):
    # 推論を実行
    results = model(image_path, save=False)

    # 画像と同じファイル名（拡張子を除く）でテキストファイル出力
    basename = os.path.splitext(os.path.basename(image_path))[0]
    save_txt_path = os.path.join(dst_label_dir, "train", f"{basename}.txt")

    # YOLO形式の推論結果をテキストファイルに書き込み
    with open(save_txt_path, 'w', encoding='utf-8') as f:
        for r in results:
            boxes = r.boxes  # YOLOv8系の出力
            for box in boxes:
                cls_id = int(box.cls[0])
                x_center, y_center, width, height = box.xywhn[0]
                # YOLO形式: class x_center y_center width height（小数点以下6桁）
                line = f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
                f.write(line)

print("すべての画像に対する推論が完了し、YOLO形式でラベルを出力しました。")


modelが未定義のため、ロードを実行します。


c:\Users\ykita\FacePhoto_instance\instanceenv\lib\site-packages\ultralytics\nn\tasks.py:634: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_locati


image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_024df9c564e854ac63c356338fae68d7e8ebd20fafde2856d07f854576d25bec.JPG: 640x640 1 Right_eye, 1 Left_eye, 776.0ms
Speed: 8.0ms preprocess, 776.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


推論中:   0%|          | 1/1390 [00:03<1:10:05,  3.03s/枚]


image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\201-20071225-9-123502_11ecdb1214665a16b19b0289b20c71d2c5a5a325f974312963bd044ddd10932f.JPG: 640x640 1 Right_eye, 1 Left_eye, 755.5ms
Speed: 5.0ms preprocess, 755.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


...

推論中:   4%|▍         | 57/1390 [00:54<17:36,  1.26枚/s]


image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\208-20170809-76-161454_d5de0e290f5b8f7ae0f92b51463bda63c8ec1aa3fbfeaa78216f5e4a42664495.jpg: 640x640 3 Left_eyes, 841.3ms
Speed: 8.0ms preprocess, 841.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


推論中:   4%|▍         | 58/1390 [00:55<18:22,  1.21枚/s]


image 1/1 C:\Users\ykita\FacePhoto_instance\inference\201-295\image\208-20170809-76-161454_e2b89c52bcfdca5daeec6065504dad0947d18c1038d03ddb71d43991af115368.jpg: 640x640 2 Left_eyes, 751.0ms
Speed: 7.0ms preprocess, 751.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


推論中:   4%|▍         | 59/1390 [00:56<18:21,  1.21枚/s]

## **CVAT仕様のフォルダ構成に**

In [13]:
import os
import glob

# 現在のlabelsフォルダのパス
dst_label_dir = r"C:\Users\ykita\FacePhoto_instance\inference\201-295\cvat\labels"

# yamlとimg_listのパスを定義
data_yaml_path = os.path.join(dst_label_dir, "..", "data.yaml")
img_list_path = os.path.join(dst_label_dir, "..", "tain.txt")

# data.yaml を書き込み
with open(data_yaml_path, 'w', encoding='utf-8') as f:
    f.write("names:\n")
    f.write("  0: Right_eye\n")
    f.write("  1: Left_eye\n")
    f.write("path: .\n")
    f.write("train: train.txt\n")

# labels_dir配下の .txt ファイルから、img_list.txt を作成
txt_files = glob.glob(os.path.join(dst_label_dir, "*.txt"))

with open(img_list_path, 'w', encoding='utf-8') as f:
    for txt_file in txt_files:
        basename = os.path.splitext(os.path.basename(txt_file))[0]
        f.write(f"data/images/train/{basename}.jpg\n")

print("data.yaml と img_list.txt が、labelsフォルダと同じ階層に作成されました。")

data.yaml と img_list.txt が、labelsフォルダと同じ階層に作成されました。
